# Create a Bucket and Lock it Down to a Specific VPC

In [24]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
s3 = boto3.Session().client(service_name='s3', region_name=region)
ec2 = boto3.Session().client(service_name='ec2', region_name=region)

In [25]:
import time

timestamp = int(time.time())

bucket_secure_name = 'bucket-secure-bucket-policy-vpc-{}'.format(timestamp)
print(bucket_secure_name)

bucket-secure-bucket-policy-vpc-1608506796


In [26]:
!aws s3 mb s3://$bucket_secure_name

make_bucket: bucket-secure-bucket-policy-vpc-1608506796


In [27]:
!aws s3 ls s3://$bucket_secure_name

In [28]:
different_vpc_id='blah'

In [29]:
all_vpcs = ec2.describe_vpcs()

vpc_id=all_vpcs['Vpcs'][0]['VpcId']

print(vpc_id)

vpc-011a72f256d2c6951


In [7]:
# Create the bucket policy
bucket_policy_deny = {
        "Version": "2008-10-17",
        "Statement": [
            {
                "Effect": "Deny",
                "Principal": "*",
                "Action": [
                    "s3:ListBucket"
                ],
                "Resource": [
                    "arn:aws:s3:::{}".format(bucket_secure_name)
                ],
                "Condition": {
                    "StringNotEquals": {
#                        "aws:sourceVpc": different_vpc_id
                        "aws:sourceVpc": vpc_id
                    }
                }
            }
        ]
    }

# Set the Deny Policy on the Bucket

In [8]:
import json
import time

response = s3.put_bucket_policy(Bucket=bucket_secure_name, 
                                Policy=json.dumps(bucket_policy_deny))

print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': 'BE51AFD354EE9AFE', 'HostId': '2guKFjoBaszoWoVImA9wkrlT3Q58xsBgLESLwHhq+qsFssaqwRZpm4DvyXgNTmHCjzR6W5Fr8Z4=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': '2guKFjoBaszoWoVImA9wkrlT3Q58xsBgLESLwHhq+qsFssaqwRZpm4DvyXgNTmHCjzR6W5Fr8Z4=', 'x-amz-request-id': 'BE51AFD354EE9AFE', 'date': 'Sun, 20 Dec 2020 23:06:05 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


# Verify `AccessDenied`

In [9]:
!aws s3 ls s3://$bucket_secure_name


An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied


# Remove the Policy

In [10]:
response = s3.delete_bucket_policy(Bucket=bucket_secure_name)
print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': 'BF4339B431D09269', 'HostId': 'gpVLdFCPjsd2YgRPjRM6bSRGu+QV4I/VHUrahp8olauGQJJv20AsqNBMPm3xtyRYGBpItbJLsNE=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'gpVLdFCPjsd2YgRPjRM6bSRGu+QV4I/VHUrahp8olauGQJJv20AsqNBMPm3xtyRYGBpItbJLsNE=', 'x-amz-request-id': 'BF4339B431D09269', 'date': 'Sun, 20 Dec 2020 23:06:37 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


# Verify Access After Removing the Policy

In [11]:
!aws s3 ls s3://$bucket_secure_name

# Remove the Bucket

In [12]:
!aws s3 rm s3://$bucket_secure_name

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}